In [1]:
!pip install --upgrade gupload

from pydrive.auth import GoogleAuth
from google.colab import auth

     |████████████████████████████████| 81 kB 3.8 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.7.10 which is incompatible.


In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/AIMLProject3

/content/drive/MyDrive/AIMLProject3


In [7]:
!pwd

/content/drive/MyDrive/AIMLProject3


In [8]:
from zipfile import ZipFile
file_name = '/content/drive/MyDrive/AIMLProject3/bdd100k_images_100k.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [9]:
from zipfile import ZipFile
file_name = '/content/drive/MyDrive/AIMLProject3/bdd100k_labels_release.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


##BDD Data Processing & Converting into YOLO Format

In [10]:
import pickle
import os
from os import listdir, getcwd
from os.path import join
import json
import csv
import torch

In [11]:
# Here we need the directory of the training images
train_images_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'
# Here we need the directory of the validation images
val_images_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'

In [12]:
print(train_images_dir)
print(val_images_dir)

/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train
/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val


In [ ]:
# Creating train and val files for training 

import pickle
import os
from os import listdir, getcwd
from os.path import join

# Here we need the directory of the training images
train_images_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'
# Here we need the directory of the validation images
val_images_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'

f = open("train.txt", "w+")

for subdirs, dirs, files in os.walk(train_images_dir):
    for filename in files:
        if filename.endswith(".jpg"):
            print("Yes")
            train_image_path = os.path.join(train_images_dir, filename)
            print(train_image_path)
            f.write(train_image_path + "\n")
f.close()


f = open("val.txt", "w+")

for subdirs, dirs, files in os.walk(val_images_dir):
    for filename in files:
        if filename.endswith(".jpg"):
            print("Yes")
            val_image_path = os.path.join(val_images_dir, filename)
            print(val_image_path)
            f.write(val_image_path + "\n")
f.close()


### Extract Labels in YOLO Format

In [14]:
#!/usr/bin/python
#f = open('image_label_txt', "w+")
import json
import csv


json_data = open('/content/drive/MyDrive/AIMLProject3/bdd100k/labels/bdd100k_labels_images_train.json')
print("----- LOADING JSON -----")
data = json.load(json_data)

# Construct a dictionary
class_dict = {}

num_of_images = len(data)
# initialze current_index
latest_class_label = 0 #Only add one when it is a new class

# List of category that need to be ignorded
list_of_category_to_ignore = ["drivable area", "lane"]


# Fill out the dictionary
for i in range(num_of_images):
        #print("[STEP 1] =============  image number i: ", i)
        #print("latest_class_label: " , latest_class_label)
        current_image = data[i]
        image_name = current_image['name']
        #print("image_name: ", image_name)
        # Create a Text file for each image
        image_label_txt = str(image_name[:-4]) + ".txt"
        #image_label_txt = str(image_name[0:100]) + ".txt"
        #print("image_label_txt: ", image_label_txt)

        f = open(image_label_txt, "w+")

        num_of_object_in_image = len(current_image['labels']) # In the type of dict

        print('num_of_object_in_image: ' , num_of_object_in_image)
        # For each object in the current image
        for j in range(num_of_object_in_image):
                #print("[STEP 2] ============== object number: ", j)
                current_object = current_image['labels'][j]
                current_object_label = current_object['category']
                #print("current_object_label:   ", current_object_label)

                if current_object_label in list_of_category_to_ignore:
                        print("The current object label is in the ignore list")
                else:
                        # Get the X-Y info
                        y1 = current_object['box2d']['y1']
                        x2 = current_object['box2d']['x2']
                        x1 = current_object['box2d']['x1']
                        y2 = current_object['box2d']['y2']
                        image_width = 1280
                        image_height = 720

                        # x-y => Center of the box2d
                        bbox_x = (x1 + x2)/2
                        bbox_y = (y1 + y2)/2

                        bbox_x_normalized = bbox_x / image_width
                        bbox_y_normalized = bbox_y / image_height

                        bbox_width = abs(x1-x2)
                        bbox_height = abs(y1-y2)

                        bbox_width_normalized = bbox_width / image_width
                        bbox_height_normalized = bbox_height / image_height

                        print("Center X-Y & Width-Height")
                        print(bbox_x, bbox_y, bbox_width, bbox_height)
                        print("NORMALIZED Center X-Y & Width-Height")
                        print(bbox_x_normalized, bbox_y_normalized, bbox_width_normalized, bbox_height_normalized)


                        # Check if current_object_label is already in the dictionary (Add )
                        #if class_dict.has_key(current_object_label):
                        if current_object_label in class_dict:
                                #print("current_object_label is already in the class_dict")
                                # show current size of class_dict
                                print("len(class_dict):  ", len(class_dict))
                        else:
                                #print("New Class: add to class_dict")
                                #print("label_to_be_added: ", current_object_label)
                                class_dict[current_object_label] = latest_class_label
                                # Update label count
                                latest_class_label = latest_class_label + 1
                                # show current size of class_dict
                                print("len(class_dict):  ", len(class_dict))

                        # Get the corresponding label from the class dict
                        class_label_code = class_dict[current_object_label]
                        print("class_label_code: ", class_label_code)
                        # Prepare the line to be written down to txt file
                        line_to_be_written = str(class_label_code) + " " + str(bbox_x_normalized) + " " + str(bbox_y_normalized) + " " + str(bbox_height_normalized) + " " + str(bbox_width_normalized)
                        print("line_to_be_written: ", line_to_be_written)
                        f.write(line_to_be_written + "\n")
                        #f = open("line_to_be_written.txt","a+")
                        #f.write(str(class_label_code) + " " + str(bbox_x_normalized) + " " + str(bbox_y_normalized) + " " + str(bbox_height_normalized) + " " + str(bbox_width_normalized) + "\n" )
                        #f.close()
        f.close()
        #print("=== Show current Class Dict Collection ====")
        print(class_dict.items())
        #input("Press Enter to continue...")



#print(" --- END OF CLASS DICT CONSTRUCT ---")
print(class_dict.items())

# Save class dictionary to text file
f = open("class_dict.txt","w")
f.write( str(class_dict) )
f.close()

# Save class dictionary to csv file
w = csv.writer(open("class_dict.csv", "w"))
for key, val in class_dict.items():
        w.writerow([key, val])


Streaming output truncated to the last 5000 lines.
NORMALIZED Center X-Y & Width-Height
0.032949506249999996 0.6432623243055555 0.06419182343749999 0.08498208194444443
len(class_dict):   10
class_label_code:  2
line_to_be_written:  2 0.032949506249999996 0.6432623243055555 0.08498208194444443 0.06419182343749999
Center X-Y & Width-Height
74.51711999999999 435.17762700000003 143.352634 99.64756200000005
NORMALIZED Center X-Y & Width-Height
0.05821649999999999 0.6044133708333334 0.11199424531249999 0.13839939166666673
len(class_dict):   10
class_label_code:  2
line_to_be_written:  2 0.05821649999999999 0.6044133708333334 0.13839939166666673 0.11199424531249999
Center X-Y & Width-Height
151.43804599999999 439.548136 132.863418 83.91373599999997
NORMALIZED Center X-Y & Width-Height
0.1183109734375 0.6104835222222222 0.10379954531249999 0.11654685555555552
len(class_dict):   10
class_label_code:  2
line_to_be_written:  2 0.1183109734375 0.6104835222222222 0.11654685555555552 0.1037995453124

In [16]:
!mv /content/drive/MyDrive/AIMLProject3/0*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

In [17]:
!mv /content/drive/MyDrive/AIMLProject3/1*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

In [18]:
!mv /content/drive/MyDrive/AIMLProject3/2*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/3*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/4*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/5*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

In [19]:
!mv /content/drive/MyDrive/AIMLProject3/6*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/7*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/8*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/9*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

In [20]:
!mv /content/drive/MyDrive/AIMLProject3/a*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/b*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/c*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/d*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

mv: cannot stat '/content/drive/MyDrive/AIMLProject3/d*.txt': No such file or directory


In [21]:
#!mv /content/drive/MyDrive/AIMLProject3/e*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/
!mv /content/drive/MyDrive/AIMLProject3/f*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/

mv: cannot stat '/content/drive/MyDrive/AIMLProject3/e*.txt': No such file or directory


### Converting validation images to YOLO format

In [22]:
import json
import csv


json_data = open('/content/drive/MyDrive/AIMLProject3/bdd100k/labels/bdd100k_labels_images_val.json')
print("----- LOADING JSON -----")
data = json.load(json_data)

# Construct a dictionary
class_dict = {}

num_of_images = len(data)
# initialze current_index
latest_class_label = 0 #Only add one when it is a new class

# List of category that need to be ignorded
list_of_category_to_ignore = ["drivable area", "lane"]


# Fill out the dictionary
for i in range(num_of_images):
        #print("[STEP 1] =============  image number i: ", i)
        #print("latest_class_label: " , latest_class_label)
        current_image = data[i]
        image_name = current_image['name']
        #print("image_name: ", image_name)
        # Create a Text file for each image
        image_label_val_txt = str(image_name[:-4]) + ".txt"
        #image_label_val_txt = str(image_name[0:100]) + ".txt"
        #print("image_label_val_txt: ", image_label_val_txt)

        f = open(image_label_val_txt, "w+")

        num_of_object_in_image = len(current_image['labels']) # In the type of dict

        print('num_of_object_in_image: ' , num_of_object_in_image)
        # For each object in the current image
        for j in range(num_of_object_in_image):
                #print("[STEP 2] ============== object number: ", j)
                current_object = current_image['labels'][j]
                current_object_label = current_object['category']
                #print("current_object_label:   ", current_object_label)

                if current_object_label in list_of_category_to_ignore:
                        print("The current object label is in the ignore list")
                else:
                        # Get the X-Y info
                        y1 = current_object['box2d']['y1']
                        x2 = current_object['box2d']['x2']
                        x1 = current_object['box2d']['x1']
                        y2 = current_object['box2d']['y2']
                        image_width = 1280
                        image_height = 720

                        # x-y => Center of the box2d
                        bbox_x = (x1 + x2)/2
                        bbox_y = (y1 + y2)/2

                        bbox_x_normalized = bbox_x / image_width
                        bbox_y_normalized = bbox_y / image_height

                        bbox_width = abs(x1-x2)
                        bbox_height = abs(y1-y2)

                        bbox_width_normalized = bbox_width / image_width
                        bbox_height_normalized = bbox_height / image_height

                        print("Center X-Y & Width-Height")
                        print(bbox_x, bbox_y, bbox_width, bbox_height)
                        print("NORMALIZED Center X-Y & Width-Height")
                        print(bbox_x_normalized, bbox_y_normalized, bbox_width_normalized, bbox_height_normalized)


                        # Check if current_object_label is already in the dictionary (Add )
                        #if class_dict.has_key(current_object_label):
                        if current_object_label in class_dict:
                                # print("current_object_label is already in the class_dict")
                                # show current size of class_dict
                                print("len(class_dict):  ", len(class_dict))
                        else:
                                #print("New Class: add to class_dict")
                                #print("label_to_be_added: ", current_object_label)
                                class_dict[current_object_label] = latest_class_label
                                # Update label count
                                latest_class_label = latest_class_label + 1
                                # show current size of class_dict
                                print("len(class_dict):  ", len(class_dict))

                        # Get the corresponding label from the class dict
                        class_label_code = class_dict[current_object_label]
                        print("class_label_code: ", class_label_code)
                        # Prepare the line to be written down to txt file
                        line_to_be_written = str(class_label_code) + " " + str(bbox_x_normalized) + " " + str(bbox_y_normalized) + " " + str(bbox_height_normalized) + " " + str(bbox_width_normalized)
                        print("line_to_be_written: ", line_to_be_written)
                        f.write(line_to_be_written + "\n")
                        #f = open("line_to_be_written.txt","a+")
                        #f.write(str(class_label_code) + " " + str(bbox_x_normalized) + " " + str(bbox_y_normalized) + " " + str(bbox_height_normalized) + " " + str(bbox_width_normalized) + "\n" )
                        #f.close()
        f.close()
        print("=== Show current Class Dict Collection ====")
        print(class_dict.items())
        #input("Press Enter to continue...")



print(" --- END OF CLASS DICT CONSTRUCT ---")
print(class_dict.items())

# Save class dictionary to text file
f = open("class_dict.txt","w")
f.write( str(class_dict) )
f.close()

# Save class dictionary to csv file
w = csv.writer(open("class_dict.csv", "w"))
for key, val in class_dict.items():
        w.writerow([key, val])

Streaming output truncated to the last 5000 lines.
class_label_code:  2
line_to_be_written:  2 0.5018166671875 0.4246872013888889 0.023437483333333377 0.006591792187500012
Center X-Y & Width-Height
649.07533 310.83728099999996 18.56248800000003 20.24998599999998
NORMALIZED Center X-Y & Width-Height
0.5070901015625 0.4317184458333333 0.014501943750000024 0.028124980555555526
len(class_dict):   10
class_label_code:  2
line_to_be_written:  2 0.5070901015625 0.4317184458333333 0.028124980555555526 0.014501943750000024
Center X-Y & Width-Height
665.106569 315.89977799999997 26.99998199999993 23.624983999999984
NORMALIZED Center X-Y & Width-Height
0.51961450703125 0.4387496916666666 0.021093735937499948 0.03281247777777775
len(class_dict):   10
class_label_code:  2
line_to_be_written:  2 0.51961450703125 0.4387496916666666 0.03281247777777775 0.021093735937499948
Center X-Y & Width-Height
693.7940495 319.27477550000003 43.87497099999996 33.749977
NORMALIZED Center X-Y & Width-Height
0.542026

#### Move the YOLO format txt files to respective VAL directory where the images resides

In [23]:
!mv /content/drive/MyDrive/AIMLProject3/b*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/
!mv /content/drive/MyDrive/AIMLProject3/c*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/
!mv /content/drive/MyDrive/AIMLProject3/f*.txt /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/

In [25]:
!ls /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/class*
!mv /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/class_dict.txt  /content/drive/MyDrive/AIMLProject3/

/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val/class_dict.txt


In [26]:
!ls /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/class*
!mv /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/class_dict.txt  /content/drive/MyDrive/AIMLProject3/

/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/class_dict.txt


### fix missing labels in train folder

In [27]:
import glob, os

#label_dir = '/home/san/Dev/dataset/bdd100k_data/bdd100k/labels/100k/train'
#image_dir = '/home/san/Dev/dataset/bdd100k_data/bdd100k/images/100k/train'

label_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'
image_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'


for label in os.listdir(label_dir):
    if label.endswith('.txt'):
        label_name = os.path.splitext(label)[0]

        # Corresponding label file name
        image_name = label_name + '.jpg'
        if os.path.isfile(image_dir + '/' + image_name) == False:
            print(" -- DELETE LABEL [Image file not found -- ]", image_name)
            label_path = label_dir + '/' + label_name + '.txt'
            print("Removing labl_path", label_path)
            os.remove(label_path)

### fix missing images in train folder

In [28]:
import glob, os

'''
Sometimes your image data set might not match with your label data set.
This code does the folowing
(1) Go through your image data set
(2) Search if the corresponding label file exist in the label data set.
(3) If not, remove current image
'''

label_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'
image_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train'

for image in os.listdir(image_dir):
    if image.endswith('jpg'):
        image_name = os.path.splitext(image)[0]

        # Corresponding label file name
        label_name = image_name + '.txt'
        if os.path.isfile(label_dir + '/' + label_name) == False:
            print(" -- DELETE IMAGE [Label file not found -- ]", image_name)
            image_path = image_dir + '/' + image_name + '.jpg'
            print("Removing image_path", image_path)
            os.remove(image_path)


 -- DELETE IMAGE [Label file not found -- ] 7a53506f-9c59128b
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/7a53506f-9c59128b.jpg
 -- DELETE IMAGE [Label file not found -- ] 5ccd45ac-62dae534
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/5ccd45ac-62dae534.jpg
 -- DELETE IMAGE [Label file not found -- ] 5b7255d9-350ca0b7
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/5b7255d9-350ca0b7.jpg
 -- DELETE IMAGE [Label file not found -- ] 66fab43d-46c5b8f2
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/66fab43d-46c5b8f2.jpg
 -- DELETE IMAGE [Label file not found -- ] 79a6c89a-342f4751
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/79a6c89a-342f4751.jpg
 -- DELETE IMAGE [Label file not found -- ] 7944f396-18fb3eba
Removing image_path /content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/train/7944f396-18fb3eba.jpg
 -- 

### fix missing lables in VAL folder

In [29]:
import glob, os

#label_dir = '/home/san/Dev/dataset/bdd100k_data/bdd100k/labels/100k/train'
#image_dir = '/home/san/Dev/dataset/bdd100k_data/bdd100k/images/100k/train'

label_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'
image_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'


for label in os.listdir(label_dir):
    if label.endswith('.txt'):
        label_name = os.path.splitext(label)[0]

        # Corresponding label file name
        image_name = label_name + '.jpg'
        if os.path.isfile(image_dir + '/' + image_name) == False:
            print(" -- DELETE LABEL [Image file not found -- ]", image_name)
            label_path = label_dir + '/' + label_name + '.txt'
            print("Removing labl_path", label_path)
            os.remove(label_path)

### fix missing images in VAL folder

In [30]:
import glob, os

'''
Sometimes your image data set might not match with your label data set.
This code does the folowing
(1) Go through your image data set
(2) Search if the corresponding label file exist in the label data set.
(3) If not, remove current image
'''

label_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'
image_dir = '/content/drive/MyDrive/AIMLProject3/bdd100k/images/100k/val'

for image in os.listdir(image_dir):
    if image.endswith('jpg'):
        image_name = os.path.splitext(image)[0]

        # Corresponding label file name
        label_name = image_name + '.txt'
        if os.path.isfile(label_dir + '/' + label_name) == False:
            print(" -- DELETE IMAGE [Label file not found -- ]", image_name)
            image_path = image_dir + '/' + image_name + '.jpg'
            print("Removing image_path", image_path)
            os.remove(image_path)